# Holmberg II X-1

In [ ]:
import time
import glob
import importlib
import tempfile
import numpy as np
import astropy
import astropy.io.fits as fits
from astropy.table import Table
from astropy.wcs import WCS
from astropy.timeseries import LombScargle

from astro import astroutils
from astro import mastrometry
from astro import mimage
from astro import mphot
from utils import mutils
from utils import mlogging 
import xarray as xr
from astro.astroutils import jd2UTCtimestamp, UTCtimestamp2jd

log = mlogging.getLogger('Holmberg')

%matplotlib notebook

In [ ]:
plate_scales = {'tek2k': .3252, 'nd12': .20325, 'Marana': .44, 'acam': 0.6}
dr_tek2k = 3.8
for x in plate_scales.values():
    dr = dr_tek2k * .3252/ x
    bw = 7 * .3252/ x
    sa = 15 * .3252/ x
    print(f'{dr} {bw} {sa}')

# plot results

In [ ]:
from icecream import ic
sums = 0
for cam in ['tek2k', 'nd12', 'acam','Marana']:
    sums += sum(sum(np.isnan(xr.open_dataarray(f'/home/canis/data/cameras/{cam}/seeing_below_3.0/cntrds_v2.nc').sel(param='sigmamag').data)))
    ic(sum(sum(np.isnan(xr.open_dataarray(f'/home/canis/data/cameras/{cam}/seeing_below_3.0/cntrds_v2.nc').sel(param='mag').data))))
    #ic(sum(sum(np.isnan(xr.open_dataarray(f'/home/canis/data/cameras/{cam}/seeing_below_3.0/cntrds_v2.nc').sel(param='mag').data))))
ic(sums)

In [ ]:
len(photresults['dmag']) 
# nd12    450  562   656  736  772 773
# tek2k   651  910   1014 1073 1107 1146
# acam    0    66    121  154  179  213
# Marana  519  1506  2156 2376 2485 2640

In [ ]:
s = set()
for fn in sorted(glob.glob('/home/canis/data/61/images/tek2k/seeing_below_2.0/*.a.nc')):
    fname = fn.split('/')[-1]
    day = int(fname[4:7])
    yr = int(fname[1:3])
    s.add(int(yr * 1000 + day))
len(s)

In [ ]:
photresults.keys()

In [ ]:
cntrds[0][20]

In [ ]:
importlib.reload(mphot)
cntrds = xr.open_dataarray('/home/canis/data/cameras/tek2k/seeing_below_2.0/cntrds_v2.nc')
photresults = mphot.photcalc(cntrds, compstar=comps, nfit=0, minfrac=0.0,sclip=3,
                         doLS=True, pmin=pmin, pmax=pmax, punit='day')

In [ ]:
def stdev(x):
    return np.nanstd(x.filled(np.nan))

def get_phot(fn, sclip, nfit, comps='optimize', scale_counts=False):
    cntrds = xr.open_dataarray(fn)
    if scale_counts:
        print('Scaling comps')
        for i in range(len(cntrds)):
            for j in range(len(cntrds[0][20])):
                cntrds[i][20][j] -= 2.5 *np.log10(1.55**2/1.3**2)
                
    print(f'Using sclip={sclip}, nfit={nfit}, comps={comps}')
    photresults = mphot.photcalc(cntrds, compstar=comps, nfit=nfit, sclip=sclip,
                             doLS=False, punit='day')
    
    ts = np.array([jd2UTCtimestamp(jd[0]) for jd in photresults['centroids'].sel(param='jd').data])
    dmags = photresults['dmagfiltered']
    dmagerrs = photresults['dmagerr']
    log.info(f'Camera sclip={sclip} filtered {len(dmags[dmags.mask==True])} values')
    log.info(f'Camera sclip={sclip} filtered {len(dmags[dmagerrs.mask==True])} err values')
    
    return ts, dmags, dmagerrs, np.array([UTCtimestamp2jd(utc) for utc in ts])

def get_mean_phot(fn, sclip, nfit, comps='optimize'):
    cntrds = xr.open_dataarray(fn)
    photresults = mphot.photcalc(cntrds, compstar=comps, nfit=nfit, sclip=sclip,
                             doLS=False, punit='day')
    
    ts = np.array([jd2UTCtimestamp(jd[0]) for jd in photresults['centroids'].sel(param='jd').data])
    dmags = photresults['dmagfiltered']
    log.info(f'Camera sclip={sclip} filtered {len(dmags[dmags.mask==True])} values')
    dmagerrs = photresults['dmagerr']
    t_mean = []
    dmag_mean = []
    dmagerr_mean  = []
    t_jd = []

    i_0 = 0
    for i in range(len(ts)):
        if ts[i][:10] == ts[i_0][:10]:
            if i == len(ts) - 1:
                t_mean.append(np.datetime64(jd2UTCtimestamp((UTCtimestamp2jd(ts[i_0]) + UTCtimestamp2jd(ts[i]))/2)))
                dmag_mean.append(np.mean(dmags[i_0:]))
                dmagerr_mean.append(np.sqrt(np.sum([s**2/(i-i_0+1)**2 for s in dmagerrs[i_0:]])))
                t_jd.append((UTCtimestamp2jd(ts[i_0]) + UTCtimestamp2jd(ts[i]))/2)
        else:
            t_mean.append(np.datetime64(jd2UTCtimestamp((UTCtimestamp2jd(ts[i_0]) + UTCtimestamp2jd(ts[i-1]))/2)))
            dmag_mean.append(np.mean(dmags[i_0:i]))
            dmagerr_mean.append(np.sqrt(np.sum([s**2/(i-i_0)**2 for s in dmagerrs[i_0:i]])))
            t_jd.append((UTCtimestamp2jd(ts[i_0]) + UTCtimestamp2jd(ts[i-1]))/2)

            i_0 = i
    return np.array(t_mean),  np.array(dmag_mean),  np.array(dmagerr_mean),  np.array(t_jd)
    # photresults_mean = photresults.copy()
    # photresults_mean['dmag'] = dmag_mean
    # photresults_mean['dmagerr'] = dmagerr_mean
    # photresults.keys()
        
def photplot(t, dmag, dmagerr, t_jd, cams, sclip, pmin, pmax, nfit,
             prefix='', outdir=None, do_LS = True, do_pdm=True, do_tsLS=False,
            interval=(-np.inf, np.inf)): # {fdir}/LS_mean_{num}.png
    errbarclr = 'gray'
    ms = 2.5
    xlab = r'$\Delta t\ \mathrm{(sec)}$'
    punit= 'day' #photresults['punit']
    xlab = r'$\mathrm{UTC}$'
    
    ##########################################################################################    
    #                                     LIGHT CURVE
    ##########################################################################################   
    
    std=np.nanstd(np.concatenate(dmag))
    t0 = np.min(np.concatenate(t_jd))
    for i in range(len(t_jd)):
        for j in range(len(t_jd[i])):
            t_jd[i][j] -= t0     
            
    mean_error = np.mean(np.concatenate(dmagerr))
   
    fig1 = plt.figure()
    plt.lineplot(dmag, t_jd,
                      'ko', ms, ylab='$\Delta m$',
                      yerrs=dmagerr, errbarcolor=errbarclr, colors=['b', 'g', 'r', 'y'],
                      inverty=True,
                      dolegend=True, labels=cams, legendpos='lower right',
                      dogrid=False, fig=fig1,
                      doyticks='left', 
                 plottitle=f'{prefix} Light curve, each camera sclip={sclip}, nfit={nfit}, stdev={round(std,3)}, mean err={mean_error}')
  
    total_t_jd = []
    total_dmag = []
    total_dmagerr = []
    
    for i in range(len(t_jd)):
        for j in range(len(t_jd[i])):
            if interval[0] < t_jd[i][j]-t0 < interval[1]:
                total_t_jd.append(t_jd[i][j])
                total_dmag.append(dmag[i][j])
                total_dmagerr.append(dmagerr[i][j])
                
    total_t_jd = np.array(total_t_jd)
    total_dmag = np.array(total_dmag)
    total_dmagerr = np.array(total_dmagerr)
    
#     from icecream import ic
#     ic(total_dmag)
    filtered = mutils.clip(total_dmag, sclip)[0]
    log.info(f'sclip={sclip} filtered {len(filtered[filtered.mask==True])} values')
    
    std = stdev(filtered)
    fig2 = plt.figure()
    plt.lineplot(filtered, total_t_jd,
                      'ko', ms, ylab='$\Delta m$',
                      yerrs=[total_dmagerr], errbarcolor=errbarclr, colors=['black'],
                      inverty=True,
                      dolegend=True, labels='all data combined', legendpos='lower right',
                      dogrid=False,
                      doyticks='left', 
                 plottitle=f'{prefix} Light curve, additional sclip={sclip}, nfit={nfit}, stdev={round(std,3)}, interval={interval}')
    
    ##########################################################################################    
    #                                     TSTOOLS LOMB SCARGLE
    ##########################################################################################     
    if do_tsLS:
        from dynamics import tstools
        nfreqs = 20000
        # 1-->0.15148718753, 2-->0.045500263896, 3-->0.002699796063,
        # 4-->0.000063342484, 5-->0.00000057330, 6-->0.00000000197
        # 2.6-->1/107, 3.3-->1/1034, 3.9-->1/10396
        thresh = [0.317, 0.0455, 0.0027, 6.33e-5, 5.73e-7]

        periodx=True
        plot_window=True


        results = tstools.lombscargle(total_t_jd, filtered, 1/pmax, 1/pmin, nfreqs, thresh, 'amplitude')
        psmag, freqs, fap, W = results
        xvals = 1/freqs if periodx else freqs

        log.info(fap)

        fig3 = plt.figure() # figsize=(12,6)
        for sigma in fap:
            plt.lineplot(data=np.linspace(sigma,sigma, 2), xvals=np.linspace(0, 10000, 2))
        plt.lineplot(data=psmag, xvals=xvals, logx=True, plottitle=f'{prefix} LS', ylim=(0,max(max(psmag), fap[[-1]])))
        if plot_window:
            fig4 = plt.figure()
            plt.lineplot(data=W, xvals=xvals, plottitle=f'{prefix} WINDOW, interval={interval}', logx=True)
    ##########################################################################################    
    #                                     ASTROPY LOMB SCARGLE
    ##########################################################################################    
    if do_LS:
        from astropy.timeseries import LombScargle
        thresh = [0.317, 0.0455, 0.0027, 6.33e-5, 5.73e-7]

        yf, tf, yerrf = ma2float(filtered, total_t_jd, total_dmagerr)
        print(len(yf))
        print(len(tf))
        print(len(yerrf))
        
        ls = LombScargle(tf, yf, yerrf)
        frequency, power = ls.autopower(minimum_frequency=1/pmax, maximum_frequency=1/pmin)
        fap = ls.false_alarm_level(thresh)

        fig5 = plt.figure()
        for x in fap:
            plt.lineplot(np.array([x, x]), np.array([0,1000]))
        plt.lineplot(power, 1/frequency, plottitle=f'{prefix} astropy LS with errors', logx=True)
        
        fig6 = plt.figure()
        ls_window = LombScargle(tf, np.ones(len(yf)) * 1000)
        frequency, power = ls_window.autopower(minimum_frequency=1/pmax, maximum_frequency=1/pmin)
        plt.lineplot(power, 1/frequency, plottitle=f'{prefix} astropy LS window', 
                     ylim=(0,0.16),  logx=True)
        
        peak = 1/frequency[np.argmax(power)]
        fig7 = plot_phase(yf, tf, yerrf, peak,title=f'{prefix} period={peak} plot')
        
        ls = LombScargle(tf, yf)
        frequency, power = ls.autopower(minimum_frequency=1/pmax, maximum_frequency=1/pmin)
        fap = ls.false_alarm_level(thresh)

        fig8 = plt.figure()
        for x in fap:
            plt.lineplot(np.array([x, x]), np.array([0,1000]))
        plt.lineplot(power, 1/frequency, plottitle=f'{prefix} astropy LS', logx=True)

        
        
    
    ##########################################################################################    
    #                                     PHASE DISPERSION MINIMIZATION
    ##########################################################################################   
    if do_pdm:
        from dynamics import tstools
        periods = np.linspace(pmax, pmin, 5000)
        tthetas = []
        pthetas= []
        dt=0.1
        binsize = 150
        for p in periods:
            ttheta = (tstools.tpdm(p, tf, yf, dt=dt, stattype='median', subhar=0))
            tthetas.append(ttheta)
            ptheta = tstools.pdm(p, tf, yf, binsize)
            pthetas.append(ptheta)
            #print(p, ttheta, ptheta)
        fig9 = plt.figure()
        plt.lineplot(np.array(tthetas), np.array(periods), 
                     plottitle=f'{prefix} tpdm: dt={dt}', logx=True)
        fig10 = plt.figure()
        plt.lineplot(np.array(pthetas), np.array(periods), 
                     plottitle=f'{prefix} pdm: binsize={binsize}', logx=True)
    
    # Save
    if outdir is not None:
        if not os.path.exists(outdir):
            os.makedirs(outdir)
        import pickle
        pickle.dump(fig1, open(f'{outdir}/fig1.plt', 'wb'))
        pickle.dump(fig2, open(f'{outdir}/fig2.plt', 'wb'))
        if do_tsLS:
            pickle.dump(fig3, open(f'{outdir}/fig3.plt', 'wb'))
            pickle.dump(fig4, open(f'{outdir}/fig4.plt', 'wb'))
        if do_LS:
            pickle.dump(fig5, open(f'{outdir}/fig5.plt', 'wb'))
            pickle.dump(fig6, open(f'{outdir}/fig6.plt', 'wb'))
            pickle.dump(fig7, open(f'{outdir}/fig7.plt', 'wb'))
            pickle.dump(fig8, open(f'{outdir}/fig8.plt', 'wb'))
        if do_pdm:
            pickle.dump(fig9, open(f'{outdir}/fig9.plt', 'wb'))
            pickle.dump(fig10, open(f'{outdir}/fig10-.plt', 'wb'))
    return yf, tf, yerrf 

def check_mask(arr, i):
    return (isinstance(arr, np.ma.MaskedArray) and arr.mask[i]==False) or not isinstance(arr, np.ma.MaskedArray)
def ma2float(y, t, yerr):
    masked_count = 0
    nan_count = 0
    yf = []
    tf = []
    yerrf = []
    for i in range(len(y)):
        if check_mask(y, i) and check_mask(yerr, i):
            if not np.isnan(y.data[i]) and not np.isnan(yerr[i]):
                yf.append(y.data[i])
                tf.append(t[i])
                yerrf.append(yerr[i])
            else:
                nan_count += 1
        else:
            masked_count += 1
    print(f'ma2float removed: {len(y) - len(yf)}, masked={masked_count}, nan={nan_count}')
    return np.array(yf), np.array(tf), np.array(yerrf)

# Plot main

In [ ]:
isinstance(x, np.ma)

In [ ]:
y, t, yerr, total_dmag = plot_main([3.0], comps='optimize', do_comp=1, save=False, intervals=[(-np.inf, np.inf)])

In [ ]:
#log.get('mphot').setLevel(log.WARNING)
def plot_main(seeings=[3.0], comp_version='_v3', phot_version='_v4', nfit=0, comps='optimize', 
              sclip=3, pmin=4, pmax=1000, do_comp=None, do_LS=True, do_pdm=False, do_tsLS=False,
             intervals=[(-np.inf, np.inf), (1400, np.inf), (0, 650), (1000, 1400)], save=True):
    for i in seeings:
        total_t = []
        total_dmag = []
        total_dmagerr = []
        total_t_jd = []

        cams = ['tek2k', 'nd12', 'Marana', 'acam']
        for c in cams:
            if do_comp is None:
                # ULX
                fn=f'/home/canis/data/cameras/{c}/seeing_below_{i}/cntrds{comp_version}.nc'
            else:
                # Comp
                fn=f'/home/canis/data/cameras/{c}/seeing_below_{i}/cntrds{comp_version}_{do_comp}.nc'
            log.info(fn)
            scale_counts = c == 'Marana' or c=='acam'
            t, dmag, dmagerr, t_jd = get_phot(fn, sclip=sclip, nfit=nfit, comps= comps,scale_counts=False)
            log.info(len(t))

    #         if len(total_t) > 1:
    #             # Intercalibrate WRT to first camera
    #             # photcalc takes care of the mean shift
    #             dmag *= stdev(total_dmag[0])/stdev(dmag)
    #             dmagerr *= stdev(total_dmag[0])/stdev(dmag)

#             if c in ['Marana', 'acam']:
#                 log.info('Scaling')
#                 dmag *= (1.55/1.3)**2
#                 dmagerr *= (1.55/1.3)**2
            dmag, t_jd, dmagerr = ma2float(dmag, t_jd, dmagerr)
            #print(dmag)
            total_t.append(t)
            total_dmag.append(dmag)
            total_dmagerr.append(dmagerr)
            total_t_jd.append(t_jd)
        #return total_dmag, total_t_jd, total_dmagerr
        
        print(np.nanstd(np.concatenate(total_dmag)))
        for interval in intervals:
            outdir=f'/home/canis/data/photresults/photresults{phot_version}'
            if comps == 'optimize':
                outdir += '_optimize'
            else:
                outdir+= '_'
                for c in comps:
                    outdir+=str(c)
            outdir += f'/{i}'
            if do_comp is not None:
                outdir += f'/comp{do_comp}'

            if interval[0] != -np.inf or interval[1] != np.inf:
                outdir+= f'/{interval[0]}-{interval[1]}'   
            
            if save:
                log.info(outdir)
            else:
                outdir=None
            
            if do_comp is None:
                prefix = f'seeing={i} ULX'
            else:
                prefix = f'seeing={i} Comp#{do_comp}'
            if interval == (-np.inf, np.inf):
                y, t, yerr = photplot(total_t,total_dmag,total_dmagerr,total_t_jd, cams, sclip=sclip,
                            pmin=pmin, pmax=pmax, nfit=nfit, prefix=prefix, interval=interval,
                            outdir=outdir, do_LS=do_LS, do_pdm=do_pdm)
            else:
                photplot(total_t,total_dmag,total_dmagerr,total_t_jd, cams, sclip=sclip,
                            pmin=pmin, pmax=pmax, nfit=nfit, prefix=prefix, interval=interval,
                            outdir=outdir, do_LS=do_LS, do_pdm=do_pdm)
        return y, t, yerr, total_dmag
    
plot_main([3.0], comps='optimize', do_tsLS=False, sclip=2.6, comp_version='_v3', phot_version='_v5', 
          do_pdm=False, do_LS=True)
#plot_main([3.0, 6.0], comps=[1,2,3,4,5], do_tsLS=False, sclip=2.6)
# for comp in [1,2,3,4,5]:
#     plot_main([3.0], comps='optimize', do_comp=comp, # intervals=[(-np.inf, np.inf)] 
#               do_LS=True, do_pdm=False, do_tsLS=False, sclip=2.6)

## Phase plot

In [ ]:
def LS(yf, tf, yerrf, pmin=4, pmax=1000,plot_data=True, plot_window=True, prefix=''):
    thresh = [0.317, 0.0455, 0.0027, 6.33e-5, 5.73e-7]
    if plot_data:
        plt.figure()
        plt.lineplot(yf, tf, 'bo', yerrs=yerrf, plottitle=f'Time series')
    if plot_window:
        fig2 = plt.figure()
        ls_window = LombScargle(tf, np.ones(len(yf)) * 1000)
        frequency, power = ls_window.autopower(minimum_frequency=1/pmax, maximum_frequency=1/pmin)
        plt.lineplot(power, 1/frequency, plottitle=f'astropy LS window', 
                     ylim=(0,0.16),  logx=True)
    ls = LombScargle(tf, yf, yerrf)
    frequency, power = ls.autopower(minimum_frequency=1/pmax, maximum_frequency=1/pmin)
    fap = ls.false_alarm_level(thresh)

    fig1 = plt.figure()
    for x in fap:
        plt.lineplot(np.array([x, x]), np.array([0,1000]))
    plt.lineplot(power, 1/frequency, plottitle=f'{prefix}astropy LS with errors, peak={1/frequency[np.argmax(power)]}', 
                 logx=True, ylim=(0, 0.16))
    return frequency[np.argmax(power)]
    
def parallel_sort(xs, ys, zs):
    order = np.argsort(xs)
    return xs[order], ys[order], zs[order]

def plot_phase(y, t, yerr, period, title=None):
    if title is None:
        title=f'period={period} plot'
    phase = np.array([(time % period)/ period for time in t])
    fig = plt.figure()
    plt.lineplot(y, phase, 'yo', plottitle=title)
    tb, yb, yerrb = bins(y, t, yerr, binsize=25)
    
    xs = np.array([(time % period)/ period for time in tb])
    print(len(xs), len(yb), len(yerrb))
    xs, ys, zs = parallel_sort(xs, yb, yerrb)
    plt.lineplot(ys, xs, colors='b', yerrs=zs)
    return fig
    
def bins(y, t, yerr, binsize=10):
    t_binned = []
    y_binned = []
    yerr_binned = []

    t_bin = []
    y_bin = []
    yerr_bin = []
    for i in range(len(t)):
        t_bin.append(t[i])
        y_bin.append(y[i])
        yerr_bin.append(yerr[i])
        if i == len(t) - 1 or len(t_bin) == binsize:
            t_binned.append(np.mean(t_bin))
            y_binned.append(np.mean(y_bin))
            yerr_binned.append(np.sqrt(np.sum([(err/len(yerr_bin))**2 for err in yerr_bin])))
            t_bin = []
            y_bin = []
            yerr_bin = []
    return np.array(t_binned), np.array(y_binned), np.array(yerr_binned)


In [ ]:
tf_s, yf_s, yerrf_s = parallel_sort(tf, yf, yerrf)
peak = LS(yf_s, tf_s, yerrf_s, plot_data=True, plot_window=True)
fig = plot_phase(yf_s, tf_s, yerrf_s, 114)
        

## Inject sinusoid/noise game

In [ ]:
yf, tf, yerrf, dmags = plot_main(seeings=[3.0], comp_version='_v3', phot_version='_v5', nfit=0, comps='optimize', 
              sclip=2.6, pmin=4, pmax=1000,
             intervals=[(-np.inf, np.inf)], save=False)

In [ ]:
rand = np.random.default_rng(42)
1 * rand.standard_normal(100)

In [ ]:
def noisy_sinusoid(tf, A, A_noise, f, phi=0, plot=False):
    rand = np.random.default_rng(42)
    y = A* np.sin(2 * np.pi * f * tf + phi) + A_noise * rand.standard_normal(len(tf))
    if plot:
        plt.figure()
        plt.lineplot(y, tf, 'ko')
    return y
def noisy(tf, A_noise, plot=False):
    rand = np.random.default_rng(42)
    y = A_noise * rand.standard_normal(len(tf))
    if plot:
        plt.figure()
        plt.lineplot(y, tf, 'ko')
    return y

LS(tf, yf, yerrf, plot_data=True, plot_window=True)
for period in [60, 25, 258,274]:
    y_fake = yf + noisy_sinusoid(tf, A=0.25*np.std(yf), A_noise = 0.1, f=1/period, plot=False)
    LS(y_fake, tf, yerrf, plot_data=False, plot_window=False, prefix=f'Injected={period}, ')
for A_noise in [0.1, 0.5, 1]:
    y_fake = yf + noisy(tf, A_noise, plot=False)
    LS(y_fake, tf, yerrf, plot_data=False, plot_window=False, prefix=f'A_noise={A_noise}, ')

# Astropy Lomb Scargle

In [ ]:
plt.figure()
plt.lineplot(yf, tf, 'ko', yerrs= [yerrf])

In [ ]:
from astropy.timeseries import LombScargle
thresh = [0.317, 0.0455, 0.0027, 6.33e-5, 5.73e-7]
#############3
pmin = 4
pmax = 200
ls = LombScargle(tf, yf, yerrf)
frequency, power = ls.autopower(minimum_frequency=1/pmax, maximum_frequency=1/pmin)
fap = ls.false_alarm_level(thresh)

plt.figure()
for x in fap:
    plt.lineplot(np.array([x, x]), np.array([0,1000]))
plt.lineplot(power, 1/frequency, plottitle=f'astropy LS with errors')

# plt.figure()
# ls_window = LombScargle(tf, np.zeros(len(yf)), yerrf)
# frequency, power = ls_window.autopower(minimum_frequency=1/pmax, maximum_frequency=1/pmin)
# plt.lineplot(power, 1/frequency, plottitle=f'astropy LS window with errors')

###########
pmin = 4
pmax = 200
ls = LombScargle(tf, yf)
frequency, power = ls.autopower(minimum_frequency=1/pmax, maximum_frequency=1/pmin)
fap = ls.false_alarm_level(thresh)

plt.figure()
for x in fap:
    plt.lineplot(np.array([x, x]), np.array([0,1000]))
plt.lineplot(power, 1/frequency, plottitle=f'astropy LS')

plt.figure()
ls_window = LombScargle(tf, np.ones(len(yf))+30)
frequency, power = ls_window.autopower(minimum_frequency=1/pmax, maximum_frequency=1/pmin)
plt.lineplot(power, 1/frequency, plottitle=f'astropy LS window', ylim=(0,0.16))
###########

# results = tstools.lombscargle(tf, yf, 1/pmax, 1/pmin, 20000, thresh, 'power')
# psmag, freqs, fap, W = results

# plt.figure()
# for x in fap:
#     plt.lineplot(np.array([x, x]), np.array([0,1000]))
# plt.lineplot(data=psmag, xvals=1/freqs, xlim=(pmin,pmax), plottitle=f'tstools LS')

# Phase dispersion minimization

In [ ]:
def ma2float(y, t, yerr):
    yf = []
    tf = []
    yerrf = []
    for i in range(len(y)):
        if y.mask[i]==False and not np.isnan(y.data[i]):
            yf.append(y.data[i])
            tf.append(t[i])
            if np.isnan(yerr[i]):
                yerrf.append(0.0000001)
            else:
                yerrf.append(yerr[i])
    print(len(y) - len(yf))
    return np.array(yf), np.array(tf), np.array(yerrf)
tf

In [ ]:
from dynamics import tstools
importlib.reload(tstools)
periods = np.linspace(200, 4, 5000)
tthetas = []
pthetas= []
dt=0.1

for p in periods:
    ttheta = (tstools.tpdm(p, tf, yf, dt=dt, stattype='median', subhar=0))
    tthetas.append(ttheta)
    ptheta = tstools.pdm(p, tf, yf, 150)
    pthetas.append(ptheta)
    print(p, ttheta, ptheta)


In [ ]:
plt.figure()
plt.lineplot(np.array(tthetas), np.array(periods), plottitle=f'tpdm: All data 3.0 seeing, dt={dt}')
plt.figure()
plt.lineplot(np.array(pthetas), np.array(periods), plottitle=f'pdm: All data 3.0 seeing, dt={dt}')

# LS Sinusoid removal game

In [ ]:
from astropy.timeseries import LombScargke
nfreqs = 60000
# 1-->0.15148718753, 2-->0.045500263896, 3-->0.002699796063,
# 4-->0.000063342484, 5-->0.00000057330, 6-->0.00000000197
# 2.6-->1/107, 3.3-->1/1034, 3.9-->1/10396
thresh = [0.317, 0.0455, 0.0027, 6.33e-5, 5.73e-7]
   
pmin=4
pmax=500
ls = LombScargle(tf, yf, yerrf)
frequency, power = ls.autopower(minimum_frequency=1/pmax, maximum_frequency=1/pmin)
fap = ls.false_alarm_level(thresh)

fig5 = plt.figure()
for x in fap:
    plt.lineplot(np.array([x, x]), np.array([0,1000]))
plt.lineplot(power, 1/frequency, plottitle=f'astropy LS with errors', logx=True)

peaks = [1/frequency[np.argmax(power)],104.7, 462.5]

# plt.figure()
# plt.lineplot(y, t, 'ko')
whitened = y_nn.copy()

results = tstools.lombscargle(t_nn, y_nn, 1/pmax, 1/pmin, nfreqs, thresh, 'amplitude')
psmag, freqs, fap, W = results

plt.figure()
plt.lineplot(data=psmag, xvals=1/freqs, xlim=(pmin,pmax), plottitle='original')
    #125 94.41 181.94 154 263.37                                                                     
    #45 40.3 50.3/4 55.6 5.23 5.37 15.188
ps = [1.996, 15.192, 5.2977, 126.35, 8.88, 45.15, 6.038, 4.173, 5.25, 4.657,
     6.884, 19.59, 15.802, 9.1658, 5.806, 4.119, 7.275, 5.38, 4.433, 32.888,
     8.7396, 4.577] # 1.3616, 5.299, 15.1885, 8.131, 3.8825, 2.25,
for p in ps: 
    whitened, A, B, m, b, fitrms = tstools.removesinusoid(t_nn, whitened, 1/p)
#     results = tstools.lombscargle(t_nn, whitened, 1/pmax, 1/pmin, nfreqs, thresh, 'amplitude')
#     psmag, freqs, fap, W = results

#     plt.figure()
#     plt.lineplot(data=psmag, xvals=1/freqs, xlim=(pmin,69), plottitle=f'rm {p}')

results = tstools.lombscargle(t_nn, whitened, 1/pmax, 1/pmin, nfreqs, thresh, 'amplitude')
psmag, freqs, fap, W = results

plt.figure()
plt.lineplot(data=psmag, xvals=1/freqs, xlim=(pmin,pmax), plottitle=f'rm {ps[-1]}')
 

# plt.figure()
# plt.lineplot(whitened,t_nn, 'ko')








In [ ]:
np.array(z).shape

In [ ]:
importlib.reload(plt)

# mplot

In [ ]:
np.random.normal(size=10)

In [ ]:
xs = np.linspace(0, 2*np.pi, 10)
errors = list(np.random.normal(size=10) * 0.1)
ys = np.zeros(10) + 0.1*np.random.standard_normal(10)
ys = list(ys)
ys[2] = 2
ys = np.array(ys)

print(ys)
filtered = mutils.clip(ys, 3)[0]

yerrs = [np.linspace(0.1, 0.3, 10)]
log.info(f'sclip={3} filtered {len(filtered[filtered.mask==True])} values')
plt.figure()
plt.lineplot(ys, xs, 'ko', yerrs=yerrs, ylim=(-0.5, 2.1))
plt.figure()
plt.lineplot(filtered, xs, 'ko', yerrs=yerrs, ylim=(-0.5, 2.1))
print(filtered)

In [ ]:
fig1, fig2 = mphot.photplot( photresults,
                             fig1=fig1 if 'fig1' in locals() else None,
                             fig2=fig2 if 'fig2' in locals() else None,
                             doLSwindow=True,
                             dateslant=False, logperiod=True )
del fig2

In [ ]:
fig = plt.figure()
plt.lineplot(np.array([1,2,3,4,5]), np.array([1,2,3,4,5]))
import pickle
pickle.dump(fig, open('FigureObject.fig.pickle', 'wb'))

In [ ]:
fig = pickle.load(open('FigureObject.fig.pickle', 'rb'))
fig.show()

Opening pickled plots

In [ ]:
import glob
import pickle
import numpy as np

from utils import mplot as plt

def show(seeing=3.0, plots=[1,2,3,4,5,6,7,8,9,10], interval=None, version=5, ext='_optimize', mean=False):
    print(f'_v{version}{ext}')
    if mean:
        fdir=f'/home/canis/data/photresults_mean_v{version}{ext}'
    else:
        fdir=f'/home/canis/data/photresults_v{version}{ext}'
    if isinstance(plots,int):
        plots = [plots]
    if plots == 'astropy':
        plots = [1, 5,6,7]
    if interval is None or (interval[0] == -np.inf and interval[1] == np.inf):
        print(interval)
        files = sorted(glob.glob(f'{fdir}/{seeing}/*.plt'))
    else:
        print(interval)
        files = sorted(glob.glob(f'{fdir}/{seeing}/{interval[0]}-{interval[1]}/*.plt'))
    i = 0
    for fn in files:
        i += 1
        if i in plots:
            fig = pickle.load(open(f'{fn}', 'rb'))
            fig.show()

def showc(seeing=3.0, plots=[1,2,3,4,5,6,7,8,9,10], c=1, interval=None, version=5, ext='_optimize'):
    print(f'_v{version}{ext}')
    fdir=f'/home/canis/data/photresults_v{version}{ext}'
    if c=='all':
        c = [1,2,3,4,5,6,7,8,9,10]
    if isinstance(plots,int):
        plots = [plots]
    if plots == 'astropy':
        plots = [1, 5,6,7]
    if interval is None or (interval[0] == -np.inf and interval[1] == np.inf):
        print(interval)
        files = sorted(glob.glob(f'{fdir}/{seeing}/comp{c}/*.plt'))
    else:
        print(interval)
        files = sorted(glob.glob(f'{fdir}/{seeing}/{interval[0]}-{interval[1]}/comp{c}/*.plt'))
    i = 0
    for fn in files:
        i += 1
        if i in plots:
            fig = pickle.load(open(f'{fn}', 'rb'))            
            fig.show()
            fig.savefig(f'/home/canis/fig{i}.png')

def close():
    plt.close('all')